1. What is the difference between mutable and immutable objects in Python?

🧠 Intuitive Overview

Imagine writing on a whiteboard vs carving on stone:

Whiteboard (Mutable): You can erase and change content anytime.

Stone (Immutable): Once carved, it can’t be changed. If you want a different text, you must create a new stone.

This is how mutable and immutable objects behave in Python.

📐 Deep Technical Explanation

Definition:

Mutable objects: Can be changed in-place after creation (their state or contents can be modified).

Immutable objects: Cannot be changed after creation — any "modification" creates a new object in memory.

Key Details:

Every object in Python has a unique id() (memory reference).

For mutable objects, modifying them preserves the same id.

For immutable objects, any modification creates a new object with a new id.

Typical categories:

Mutable	Immutable
list, dict, set, bytearray	int, float, str, tuple, frozenset, bytes

🌍 Real-World Example / Case Study

In Google’s search ranking pipelines (written partly in Python), large intermediate datasets are stored in lists (mutable) for in-place transformations to save memory and avoid object creation overhead.

Immutable types (like strings) are used for dictionary keys or cache lookups in systems like Amazon’s recommendation engines because they are hashable and safe to use as keys (mutable objects are not hashable).

⚖️ Comparison with Alternatives
Aspect	                        Mutable	                    Immutable
Memory efficiency	            In-place changes	        New object created each time
Thread-safety	                Risky (race conditions)	    Safer (no change possible)
Hashability	                    Usually not hashable	    Always hashable
Performance	                    Faster updates	            Safer in caching & parallelism

Use mutable when you need frequent modifications (lists, dynamic data).

Use immutable for fixed keys, caching, concurrency, and ensuring data integrity.

In [1]:
nums = [1, 2, 3]
print(id(nums))       # e.g. 140347299392768
nums.append(4)
print(nums)           # [1, 2, 3, 4]
print(id(nums))       # same ID as before -> modified in-place


4703844864
[1, 2, 3, 4]
4703844864


In [2]:
name = "Alice"
print(id(name))       # e.g. 140347299391600
name += " Smith"
print(name)            # "Alice Smith"
print(id(name))        # new ID -> new object created


4703663952
Alice Smith
4703889904


2. How is memory management handled in Python?

🧠 Intuitive Overview

Think of Python as having a “memory housekeeping staff”:

Every time you create an object (like a list or string), it’s stored in a big warehouse (heap memory).

Each object has a tag showing how many people are still using it (reference count).

When no one needs it anymore (reference count = 0), the garbage collector comes and sweeps it away to free space.

Occasionally, the staff also goes through the warehouse and removes forgotten trash in cycles.

This is automatic memory management in Python.

📐 Deep Technical Explanation
🧮 Memory Allocation

Python uses a private heap for all objects and data structures.

Memory allocation is managed by:

Python Memory Manager: handles object allocation on the heap.

PyMalloc: optimized allocator for small objects (<512 bytes) to reduce OS calls.

Operating System (OS): provides raw memory blocks to Python when needed.

🧠 Reference Counting

Every object has an internal counter: ob_refcnt

On assignment, refcount += 1; on object deletion or scope exit, refcount -= 1

When refcount == 0, the memory is freed immediately.

🧹 Garbage Collection (GC)

Reference counting alone fails for circular references (A → B → A)

Python garbage collector (cyclic GC) periodically:

Tracks container objects (lists, dicts, classes)

Detects reference cycles using a generational GC algorithm (Gen0, Gen1, Gen2)

Collects unreachable cycles

Generational Hypothesis: Young objects die quickly, so they’re collected more often.

📦 Memory Deallocation

del obj decrements reference count, may free memory

Actual memory return to OS is delayed (managed by Python’s memory allocator, not directly by the OS)



            +-----------+
            |   Python  |
            |  Program  |
            +-----------+
                   |
          +---------------------+
          | Python Memory Heap |
          +---------------------+
        |           |            |
   PyMalloc     RefCount      GC (cycles)


🌍 Real-World Example / Case Study

At Instagram (built on Python), memory bloat from cyclic references in long-lived processes caused crashes.

Engineers used gc.set_threshold() to tune garbage collection frequency, and rewrote memory-heavy logic to break cycles and use weak references (weakref) for caches.

⚖️ Comparison With Alternatives
Language	    Memory Mgmt	                        Characteristics
Python	      Automatic (ref counting + GC)	      Slower, easy, safe
Java	        GC only	                            No refcount overhead but unpredictable pause
C++	          Manual (RAII/smart pointers)	      Fast, deterministic but error-prone

Tradeoff: Python prioritizes developer productivity over raw performance.

In [3]:
import sys
a = [1, 2, 3]
print(sys.getrefcount(a))  # e.g. 2 (one from 'a', one from getrefcount arg)


2


In [4]:
import gc
gc.collect()  # manually trigger collection


0

3. Explain the difference between is and == operators with examples.

🧠 Intuitive Overview

Think of two books:

They might be two copies of the same novel (same content) → == says they are equal.

But they are physically different books on different shelves → is says they are not the same object.

In Python terms:

== checks if two objects have the same value/content.

is checks if two variables refer to the same object in memory (identity).

📐 Deep Technical Explanation
Operator	Meaning                 	Underlying Mechanism
==	        Equality of values	        Calls __eq__() method of the object
is	        Identity (same object)	    Compares id(obj1) == id(obj2)

== can be overridden in custom classes by defining __eq__.

is cannot be overridden; it’s based purely on object identity (memory address).

🌍 Real-World Example / Case Study

In Google’s internal Python config tools, is is used to compare against sentinel objects (unique markers like NOT_SET) because identity checks are O(1) and safe.

For user data or values (like strings, numbers, lists), == is used because it checks semantic equality.

| Aspect      | `is`                            | `==`                            |
| ----------- | ------------------------------- | ------------------------------- |
| Checks      | Identity (same object)          | Value (same content)            |
| Speed       | Faster (just compares pointers) | May be slower (deep comparison) |
| Overridable | No                              | Yes (`__eq__`)                  |
| Use case    | Sentinel/singletons (`None`)    | Comparing actual data values    |



In [6]:
#== cheks for value equality

a = [1, 2, 3]
b = [1, 2, 3]

print(a == b)   # True (values same)
print(a is b)   # False (different objects)


True
False


In [7]:
#is checks for identity (same object in memory)

x = "hello"
y = "hello"

print(x == y)   # True
print(x is y)   # May be True (string interning) but not guaranteed


True
True


In [5]:
# identity confirmed via id()
a = 10000
b = 10000

print(id(a), id(b))  # Different numbers -> different objects


4702522928 4702523792


In [9]:
#always use 'is' to check for None
x = None

if x is None:   # ✅ correct
    pass
if x == None:   # ⚠️ not recommended
    pass


4. What is the difference between a deep copy and a shallow copy in Python?

🧠 Intuitive Overview

Imagine copying a box full of folders:

Shallow Copy → You make a new box, but just put the same folders inside it. Both boxes share the same   folders.

Deep Copy → You make a new box and also duplicate each folder and its contents. The two boxes are completely independent.

In Python terms:

Shallow copy creates a new outer object but references the same inner objects.

Deep copy creates a new outer object and recursively copies all inner objects.

📐 Deep Technical Explanation

Shallow Copy

Creates a new container object.

Copies references (pointers) to the same child objects.

Changes to mutable children affect both copies.

Deep Copy

Creates a new container object.

Recursively creates new copies of all children.

Completely independent clone.

Modules Used

copy module provides:

copy.copy(obj) → shallow copy

copy.deepcopy(obj) → deep copy

🌍 Real-World Example / Case Study

At Google, configuration dictionaries are often shallow-copied when passing to functions to avoid accidental global state changes but still share large immutable substructures (efficient).

At Amazon, product catalog structures are deep-copied before transformation pipelines to ensure data isolation and avoid unintended mutations between concurrent processing threads.

| Aspect        | Shallow Copy                       | Deep Copy                   |
| ------------- | ---------------------------------- | --------------------------- |
| Outer object  | New                                | New                         |
| Inner objects | Shared (same references)           | Cloned recursively          |
| Speed         | Faster                             | Slower                      |
| Memory usage  | Lower                              | Higher                      |
| Use when      | Want structure copy but share data | Want full independent clone |




In [10]:
import copy

original = [[1, 2], [3, 4]]

shallow = copy.copy(original)
deep = copy.deepcopy(original)

# Modify inner element
original[0][0] = 99

print(original)  # [[99, 2], [3, 4]]
print(shallow)   # [[99, 2], [3, 4]]  <-- affected
print(deep)      # [[1, 2], [3, 4]]   <-- unaffected


[[99, 2], [3, 4]]
[[99, 2], [3, 4]]
[[1, 2], [3, 4]]


In [ ]:
# Ids show reference sharing 

print(id(original[0]) == id(shallow[0]))  # True
print(id(original[0]) == id(deep[0]))     # False


True
False
